In [ ]:
!pip install pymysql
!pip install sqlalchemy

In [ ]:
!pip install --upgrade pip

In [ ]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input

password = getpass.getpass()
bd = "spotify_db" # select your name for the whole database
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+bd  #on this step you have to input your MySQL Workbench password
engine = create_engine(connection_string)
engine
# df_release = pd.read_csv("/Users/leonb/OneDrive/Desktop/Spotify DB/sp_release.csv", encoding="utf-8")
# df_release.to_sql("sp_release", con=engine, if_exists="append", index=False)
# df_artist = pd.read_csv("/Users/leonb/OneDrive/Desktop/Spotify DB/sp_artist.csv", encoding="utf-8")
# df_artist.to_sql("sp_artist", con=engine, if_exists="append", index=False)
df_track = pd.read_csv("/Users/leonb/OneDrive/Desktop/Spotify DB/sp_track.csv", encoding="utf-8")
df_track.to_sql("sp_track", con=engine, if_exists="append", index=False)
# df_artist_release = pd.read_csv("/Users/leonb/OneDrive/Desktop/Spotify DB/sp_artist_release.csv", encoding="utf-8")
# df_artist_release.to_sql("sp_artist_release", con=engine, if_exists="append", index=False)
# df_artist_track = pd.read_csv("/Users/leonb/OneDrive/Desktop/Spotify DB/sp_artist_track.csv", encoding="utf-8")
# df_artist_track.to_sql("sp_artist_track", con=engine, if_exists="append", index=False)
# audio_features = pd.read_csv("/Users/leonb/OneDrive/Desktop/Spotify DB/audio_features.csv", encoding="utf-8")
# audio_features.to_sql("audio_features", con=engine, if_exists="append", index=False)


In [ ]:
!pip install mysql.connector-python

In [13]:
import mysql.connector
from mysql.connector import Error

def create_indexes():
    try:
        # Connect to MySQL database
        connection = mysql.connector.connect(
            host='localhost',  # Change this to your MySQL host
            user='root',  # Change this to your MySQL user
            password='Drachenritter8',  # Change this to your MySQL password
            database='spotify_db',  # Change this to your database
            connect_timeout=900  # Set a longer timeout to prevent timeouts
        )

        if connection.is_connected():
            cursor = connection.cursor()
            
            # Index creation queries
            index_queries = [
                "CREATE INDEX idx_artist_covering ON sp_artist(artist_id, artist_name);"
            ]

            for query in index_queries:
                cursor.execute(query)
                print(f"Executed: {query}")

            connection.commit()
            print("Indexes created successfully.")

    except Error as e:
        print(f"Error: {e}")
    
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection closed.")

if __name__ == "__main__":
    create_indexes()


Executed: CREATE INDEX idx_artist_covering ON sp_artist(artist_id, artist_name);
Indexes created successfully.
MySQL connection closed.


In [27]:
import mysql.connector
from mysql.connector import Error

# Connect to MySQL
def create_connection():
    try:
        connection = mysql.connector.connect(
            host="localhost",  # Your MySQL host
            user="root",       # Your MySQL user
            password="Drachenritter8",  # Your MySQL password
            database="spotify_db"  # The database you're working with
        )
        if connection.is_connected():
            print("Connected to MySQL database")
        return connection
    except Error as e:
        print(f"Error: {e}")
        return None

In [29]:
def get_missing_isrc(connection):
    try:
        cursor = connection.cursor()
        query = """
        SELECT st.isrc
        FROM sp_track st
        LEFT JOIN audio_features af ON st.isrc = af.isrc
        WHERE af.isrc IS NULL;
        """
        cursor.execute(query)
        missing_isrc = cursor.fetchall()
        cursor.close()
        return missing_isrc
    except Error as e:
        print(f"Error: {e}")
        return []

In [30]:
def insert_missing_isrc(connection, missing_isrc, batch_size=1000):
    try:
        cursor = connection.cursor()
        query = "INSERT IGNORE INTO audio_features (isrc) VALUES (%s)"  # Use INSERT IGNORE to skip duplicates
        
        # Batch insertion
        for i in range(0, len(missing_isrc), batch_size):
            batch = missing_isrc[i:i + batch_size]
            cursor.executemany(query, batch)
            connection.commit()  # Commit after each batch
            print(f"Inserted {len(batch)} rows.")
        
        cursor.close()
    except Error as e:
        print(f"Error: {e}")

In [31]:
def main():
    connection = create_connection()
    if connection:
        # Step 1: Get the missing ISRC values
        missing_isrc = get_missing_isrc(connection)
        if missing_isrc:
            # Step 2: Insert missing ISRC values in batches
            insert_missing_isrc(connection, missing_isrc, batch_size=1000)
        else:
            print("No missing ISRC values to insert.")
        connection.close()

if __name__ == "__main__":
    main()

Connected to MySQL database
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 1000 rows.
Inserted 100

In [10]:
import pymysql

# Connect to MySQL
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="Drachenritter8",
    database="spotify_db"
)

try:
    with connection.cursor() as cursor:
        # Run multiple queries (SET and ALTER TABLE)
        cursor.execute("""
            SET foreign_key_checks = 0;
        """)
        
        cursor.execute("""
            ALTER TABLE sp_track
            ADD CONSTRAINT fk_track_release_id
            FOREIGN KEY (release_id) REFERENCES sp_release(release_id)
            ON DELETE CASCADE
            ON UPDATE CASCADE;
        """)

        cursor.execute("""
            SET foreign_key_checks = 1;
        """)
        
    # Commit changes
    connection.commit()
    print("✅ Foreign key added successfully!")

except pymysql.MySQLError as e:
    print(f"❌ Error: {e}")

finally:
    connection.close()

✅ Foreign key added successfully!
